In [4]:
from scipy.integrate import odeint
from numpy.random import randn, rand, uniform
from time import time
import numpy as np
import pylab as pl
import lib
import os

num_e = 200
num_i = 50
sigma_e = 0.05
i_ext_e = 1.4 * np.ones(num_e) * (1 + sigma_e * randn(num_e))
sigma_i = 0.00
i_ext_i = 0.0 * np.ones(num_i) * (1 + sigma_i * randn(num_i))
g_hat_ee = 0
g_hat_ei = 0.25
g_hat_ie = 0.25
g_hat_ii = 0.25
p_ee = 0.5
p_ei = 0.5
p_ie = 0.5
p_ii = 0.5

v_rev_e = 0.0
v_rev_i = -75.0
tau_r_e = 0.5
tau_peak_e = 0.5
tau_d_e = 3.0
tau_r_i = 0.5
tau_peak_i = 0.5
tau_d_i = 9.0
t_final = 200.0
dt = 0.02

spikeThreshold = -20
# Process network parameters a bit:

u_ee = rand(num_e, num_e)
u_ei = rand(num_e, num_i)
u_ie = rand(num_i, num_e)
u_ii = rand(num_i, num_i)
g_ee = (g_hat_ee * (u_ee < p_ee) / (num_e * p_ee)).T
g_ei = (g_hat_ei * (u_ei < p_ei) / (num_e * p_ei)).T
g_ie = (g_hat_ie * (u_ie < p_ie) / (num_i * p_ie)).T
g_ii = (g_hat_ii * (u_ii < p_ii) / (num_i * p_ii)).T

"""
Consider, for example, the i-th e-cell and the j-th i-cell. the
probability that there is a synaptic connection at all from the
i-th e-cell to the j-th i-cell is p_ei. if there is such a
connection, its strength is g_hat_ei/(num_e*p_ei). Note that
num_e*p_ei is the expected number of excitatory inputs into an
inhibitory cell. Therefore dividing by this quantity has the
effect that the expected value of the total excitatory
conductance affecting an inhibitory cell is g_hat_ei. 
"""

# ------------------------------------------------------------------#


if __name__ == "__main__":

    start = time()

    # os.setenv('OMP_NUM_THREADS', '1')
    # os.environ["OMP_NUM_THREADS"] = "1"

    initialVec = lib.splayState(i_ext_e,
                                rand(num_e),
                                lib.derivative)

    v_e = initialVec[:, 0]
    m_e = lib.m_e_inf(v_e)
    h_e = initialVec[:, 1]
    n_e = initialVec[:, 2]
    q_e = np.zeros(num_e)
    s_e = np.zeros(num_e)

    v_i = -75.0 * np.ones(num_i)
    m_i = lib.m_i_inf(v_i)
    h_i = lib.h_i_inf(v_i)
    n_i = lib.n_i_inf(v_i)
    q_i = np.zeros(num_i)
    s_i = np.zeros(num_i)


    initialConditions = np.hstack((v_e, h_e, n_e, q_e, s_e,
                                   v_i, h_i, n_i, q_i, s_i))
    t = np.arange(0, t_final, dt)
    sol = odeint(lib.derivativePopulation,
                 initialConditions,
                 t)

    lfp = np.mean(sol[:, :num_e], axis=1)

    t_e_spikes = []
    t_i_spikes = []
    for i in range(num_e):
        ts_e = lib.spikeDetection(t, sol[:, i], spikeThreshold)
        t_e_spikes.append(ts_e)
    index = 5 * num_e
    for i in range(index, index + num_i):
        ts_i = lib.spikeDetection(t, sol[:, i], spikeThreshold)
        t_i_spikes.append(ts_i)

    lib.display_time(time() - start)

    lib.spikeToFile(t_e_spikes, "t_e_spikes.txt")
    lib.spikeToFile(t_i_spikes, "t_i_spikes.txt")
    np.savetxt("lfp.txt", np.column_stack((t, lfp)), fmt="%18.6f")


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
import numpy as np
import pylab as pl
from lib import read_from_file
from main import num_e, num_i

t_e_spikes = read_from_file("t_e_spikes.txt")
t_i_spikes = read_from_file("t_i_spikes.txt")
lfp = np.loadtxt("lfp.txt", dtype=float)

fig, ax = pl.subplots(2, figsize=(10, 8), sharex=True)

for i in range(num_i):
    ax[0].plot(t_i_spikes[i], [i] * len(t_i_spikes[i]), "b.")

for i in range(num_i, num_i + num_e):
    ax[0].plot(t_e_spikes[i - num_i], [i] *
            len(t_e_spikes[i - num_i]), "r.")
t = lfp[:, 0]
lfp = lfp[:, 1]

ax[1].plot(t, lfp, lw=2, color="k")

ax[1].set_xlabel("time [ms]", fontsize=18)
ax[0].set_ylabel("neuron #", fontsize=18)
ax[1].set_ylabel("mean(v), E-cells", fontsize=18)

for i in range(2):
    ax[i].tick_params(labelsize=14)
ax[1].set_ylim(-100, 50)
ax[1].set_xlim(0, np.max(t))
pl.tight_layout()
pl.savefig("fig.png")
pl.show()

